# Text Generation

## Introduction

Markov chains can be used for very basic text generation. Think about every word in a corpus as a state. We can make a simple assumption that the next word is only dependent on the previous word - which is the basic assumption of a Markov chain.

Markov chains don't generate text as well as deep learning, but it's a good (and fun!) start.

## Select Text to Imitate

In this notebook, we're specifically going to generate text in the style of Ali Wong, so as a first step, let's extract the text from her comedy routine.

In [107]:
# Read in the corpus, including punctuation!
import pandas as pd

data = pd.read_pickle('corpus.pkl')
data

,transcript
AlonsoPraisesAMR,Fernando Alonso was delighted with his drive t...
HamiltonsCarIssues,Lewis Hamilton was left to rue the struggles h...
LeclercsPoorQualifying,Charles Leclerc enjoyed a strong drive to P4 i...
PerezOnHisFuture,Sergio Perez expects his Red Bull and F1 futur...
SainzOnHisFuture,Carlos Sainz feels that now is the time to “sp...
TotoOnMercedes,Toto Wolff has admitted that Mercedes are in a...
TsunodaPoints,Yuki Tsunoda was full of relief after scoring ...


In [108]:
# Extract only PerezOnHisFuture's text
perez_text = data.transcript.loc['PerezOnHisFuture']
perez_text[:200]

'Sergio Perez expects his Red Bull and F1 future to be clarified within the next month, with the Mexican out of contract next year. The Mexican has been in this exact position before – originally signi'

## Build a Markov Chain Function

We are going to build a simple Markov chain function that creates a dictionary:
* The keys should be all of the words in the corpus
* The values should be a list of the words that follow the keys

In [109]:
from collections import defaultdict

def markov_chain(text):
    '''The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text.'''
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = defaultdict(list)
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)

    # Convert the default dict back into a dictionary
    m_dict = dict(m_dict)
    return m_dict

In [110]:
# Create the dictionary for Ali's routine, take a look at it
perez_dict = markov_chain(perez_text)
perez_dict

{'Sergio': ['Perez'],
 'Perez': ['expects',
  'is',
  'signed',
  'still',
  'is',
  'says',
  'has',
  'hails',
  'very'],
 'expects': ['his'],
 'his': ['Red',
  'contract.',
  'drive',
  'future.',
  'F1',
  'ambition',
  'career',
  'performances',
  'future',
  'opening',
  'F1',
  'team'],
 'Red': ['Bull',
  'Bull',
  'Bull',
  'Bull',
  'Bull',
  'Bull,',
  'Bull',
  'Bull,',
  'Bull',
  'Bull'],
 'Bull': ['and', 'for', 'retained', 'had', 'family', 'in', 'family.', 'seat'],
 'and': ['F1',
  'not',
  'get',
  'making',
  'a',
  'Fernando',
  'Perez’s',
  'now',
  'is',
  'whatever',
  'the',
  'not'],
 'F1': ['future', 'future', 'after', 'and', 'career'],
 'future': ['to', 'in', 'at', 'holds.'],
 'to': ['be',
  'extend',
  'see',
  'finish',
  'go',
  'P2',
  'be',
  'really',
  'Ferrari',
  'pretty',
  'secure',
  'lose”',
  'team',
  'stay'],
 'be': ['clarified', 'long', 'a', 'a'],
 'clarified': ['within'],
 'within': ['the', 'a'],
 'the': ['next',
  'Mexican',
  'right',
  'eve

## Create a Text Generator

We're going to create a function that generates sentences. It will take two things as inputs:
* The dictionary you just created
* The number of words you want generated

Here are some examples of generated sentences:

>'Shape right turn– I also takes so that she’s got women all know that snail-trail.'

>'Optimum level of early retirement, and be sure all the following Tuesday… because it’s too.'

In [111]:
import random

def generate_sentence(chain, count=8):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the number of words you would like to see in your generated sentence.'''

    # Capitalize the first word
    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()

    # Generate the second word from the value list. Set the new word as the first word. Repeat.
    for i in range(count-1):
        word2 = random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2

    # End it with a period
    sentence += '.'
    return(sentence)

In [112]:
generate_sentence(perez_dict)

'Japan. “My main focus is always the talking.'

### Assignment:
1. Generate sentence for other comedians also.
2. Try making the generate_sentence function better. Maybe allow it to end with a random punctuation mark or end whenever it gets to a word that already ends with a punctuation mark.

In [113]:
# Extract text for other comedians
Article = ["TotoOnMercedes","AlonsoPraisesAMR","PerezOnHisFuture","SainzOnHisFuture","LeclercsPoorQualifying",
           "HamiltonsCarIssues","TsunodaPoints"] 
# Dictionary to store Markov chain for each comedian
dicts = {}

# Iterate over comedian names
for name in Article:
    article_text = data.transcript.loc[name]  # Assuming the data DataFrame contains transcripts for all comedians
    dicts[name] = markov_chain(article_text)

# Generate sentences for other comedians
generated_sentences = {}

for article, chain in dicts.items():
    generated_sentences[article] = generate_sentence(chain)

# Print generated sentences for each comedian
for article, sentence in generated_sentences.items():
    print(f"{article} says: {sentence}")


TotoOnMercedes says: Remarked after what he felt went well. “The.
AlonsoPraisesAMR says: Yeah, it and P10, and we still need.
PerezOnHisFuture says: Pretty relaxed figure when asked about potential moves.
SainzOnHisFuture says: Say is the Saudi Arabian Grand Prix “The.
LeclercsPoorQualifying says: Advantage. While he looked to P4 drive in.
HamiltonsCarIssues says: Terms of the Red Bull, and you’ve seen.
TsunodaPoints says: Getaway in five-car pit stop “[I’m] relieved,” joked.


In [114]:
import random
import string
punct=['.','!','?']

def generate_sentence(chain, max_words=15, end_with_punctuation=True):
    '''Generate a sentence using a Markov chain dictionary.
    
    Args:
    - chain: Markov chain dictionary
    - max_words: Maximum number of words in the generated sentence
    - end_with_punctuation: Whether to end the sentence with a punctuation mark
    
    Returns:
    - A generated sentence
    '''
    
    # Function to check if a word ends with punctuation
    def ends_with_punctuation(word):
        return word[-1] in punct
    
    # Reference to original function
    def original_generate_sentence(chain, count=15):
        '''Original version of generate_sentence function.'''
        word1 = random.choice(list(chain.keys()))
        sentence = word1.capitalize()
        for i in range(count-1):
            word2 = random.choice(chain[word1])
            word1 = word2
            sentence += ' ' + word2
        sentence += '.'
        return sentence
    
    # Capitalize the first word
    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()
    
    # Generate the second word from the value list. Set the new word as the first word. Repeat.
    word_count = 1
    while word_count < max_words:
        word2 = random.choice(chain[word1])
        sentence += ' ' + word2
        word_count += 1
        
        # Check if the word ends with punctuation
        if ends_with_punctuation(word2):
            break
        
        word1 = word2
    
    # End it with a punctuation mark if specified
    if end_with_punctuation:
        if not ends_with_punctuation(sentence):
            sentence += '.'
        
    return sentence.strip()

# Example usage
generated_sentence = generate_sentence(perez_dict)
print(generated_sentence)


2025 ignited the driver who wants to extend his performances do the driver who could.


In [115]:
# Extract text for other comedians
Article = ["TotoOnMercedes","AlonsoPraisesAMR","PerezOnHisFuture","SainzOnHisFuture","LeclercsPoorQualifying",
           "HamiltonsCarIssues","TsunodaPoints"] 
# Dictionary to store Markov chain for each comedian
dicts = {}

# Iterate over comedian names
for name in Article:
    article_text = data.transcript.loc[name]  # Assuming the data DataFrame contains transcripts for all comedians
    dicts[name] = markov_chain(article_text)

# Generate sentences for other comedians
generated_sentences = {}

for article, chain in dicts.items():
    generated_sentences[article] = generate_sentence(chain)

# Print generated sentences for each comedian
for article, sentence in generated_sentences.items():
    print(f"{article} says: {sentence}")


TotoOnMercedes says: Monday MORNING DEBRIEF: Who nailed their performance later on, team mate Russell past Race day.
AlonsoPraisesAMR says: Ultimately helped Alonso happy for differing tyre calls?
PerezOnHisFuture says: Get outs for 2025, there has admitted his future.
SainzOnHisFuture says: Best options for next year and Leclerc respond to “speed up” negotiations and Band of.
LeclercsPoorQualifying says: Lap was overtaken by fans.
HamiltonsCarIssues says: After the grid.  This feature is never what we need to provide consent to rue.
TsunodaPoints says: “[i’m] relieved,” joked Tsunoda crosses the RB driver is not that and RB come Abu.
